# Making a CSV data file line by line or dataframe row by row
## A dataset for Norwegian preterites

by Koenraad De Smedt at UiB

---
This notebook demonstrates how to construct and write a CSV file line by line, or to make a pandas dataframe row by row, by processing data from each line in an input file.

1.  Extract positional attributes from words
2.  Fill in missing positional attributes
3.  Writing a file line by line by reading and processing another file line by line
4.  Make a pandas dataframe row by row by reading and processing a file line by line
4.  Break up a problem in smaller steps with helper functions.

CSV is an often used format in data science, so it is useful to have some experience in converting data to this format. The current demo results in a file that can be fed into a machine learning program like [Weka](https://waikato.github.io/weka-wiki/downloading_weka/).

The examples in this notebook are based on a plain [list of Norwegian preterites, one on each line](https://git.app.uib.no/desmedt/teaching/-/raw/main/vpret-et-te.txt). The last two letters of each verbform make up the suffix (*et* or *te*), which will be the last attribute on a line in the CSV file. The three letters before the suffix (the *antepenultimate*, *penultimate* and *final* letters of the stem) will become separate attributes. All attributes will be delimited by a separator (usually a comma). Example:

>`abonnerte` -> `n,e,r,te`

>`adlet` -> `a,d,l,et`

If a preterite is shorter than the required 5 characters, a filler character should fill the places of the missing features.

>`aget'` -> `+,a,g,et`

>`ået'` -> `+,+,å,et`


## Define variables and helper functions

First define some *global* variables. This makes it easy to change them if the need should arise. The filler and separator should be non-word characters.

In [ ]:
filler = '+'
sep = ','

Now break down the problem and define some helper functions. Step one is to write a helper function that ‘fills up’ a word with fillers if the word is shorter than five letters, otherwise the word is returned as it was.

In [ ]:
def fill_pret (verb):
  if len(verb) < 5:
    return (5 - len(verb)) * filler + verb
  else: return verb

print(fill_pret('ået'))
print(fill_pret('aget'))
print(fill_pret('adlet'))

Step two is to write the main conversion function that makes one output line. This function uses the helper function and then joins the necessary features into a string with commas.

In [ ]:
def convert_pret (verb):
  verb = fill_pret(verb)
  return sep.join(verb[-5:-1]) + verb[-1]

print(convert_pret('ået'))
print(convert_pret('aget'))
print(convert_pret('adlet'))

## Convert input file to output file

Now we are ready to apply the conversion to every line that is read from a file and to write the results to another file. The input file is remotely available. If you are using Colab, the output will be a temporary file (downloadable).

In [ ]:
import requests
vpret_url = 'https://git.app.uib.no/desmedt/teaching/-/raw/main/vpret-et-te.txt'

Open the url as a stream from which you can read lines. With an open output file, first print a header (column names). Then iterate over lines from the stream, and print the converted lined to the output file. The `.strip` function strips whitespace (including newlines) from the beginning and end of each input line.

In [ ]:
vpret_stream = requests.get(vpret_url, stream=True)

with open('vpret.csv', 'w') as outfile:
  # first write the header
  print(sep.join(['ant','pen','fin','suffix']), file=outfile)
  # now iterate over input lines and write output lines
  for line in vpret_stream.iter_lines(decode_unicode=True):
    print(convert_pret(line.strip()), file=outfile)

Check the contents of the resulting file. Read and print only the first 100 characters.

In [ ]:
with open('vpret.csv') as f:
  print(f.read(100))

Test reading the file into a pandas dataframe.

In [ ]:
import pandas as pd
df_vpret = pd.read_csv('vpret.csv')
df_vpret

Download this file from Colab if necessary, or write it to Google Drive or whatever.


# Alternative: Build a dataframe row by row

First make a dataframe with four named but empty columns.

In [ ]:
import pandas as pd
df_vpret = pd.DataFrame({'ant':[], 'pen':[], 'fin':[], 'suffix':[]})
df_vpret

Define a function to make a list with the information for one row.

In [ ]:
def makerow (verb):
  filled = fill_pret(verb)
  return list(filled[-5:-2]) + [filled[-2:]]

makerow('ået')

Now iterate over the lines of the input file and add rows to the dataframe one by one, incrementing the index each time after adding a row.

In [ ]:
vpret_stream = requests.get(vpret_url, stream=True)

i = 0
for line in vpret_stream.iter_lines(decode_unicode=True):
  df_vpret.loc[i] = makerow(line.strip())
  i += 1

df_vpret

Now the dataframe can be used immediately for further processing, or it can be written to file as in the following cell. Let's not write the row names.

In [ ]:
df_vpret.to_csv('vpret.csv', index=False)

### Exercises

1.   Use .value_count() to find out if suffix *te* or *et* is most frequent.
2.   Change the filler and separator. Run the program and check the result.
3.   If you want to use 6 letters, for instance, instead of 5, you have to change the program in several places. This may be inconvenient and prone to errors. A better way is to make a global variable `nletters` for the number of letters of the word that will be used in the output. Add this variable at the beginning and adapt the program where necessary to use this variable.